# Looking at https://github.com/ytsvetko/metaphor

In [92]:
##all the imports...
#%pylab inline
import re
import math
import string
import nltk
import pprint
import matplotlib
import pandas as pd
import numpy as np
from nltk import word_tokenize
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
from collections import Counter
from __future__ import division
from nltk.collocations import *
import string, random
from nltk.corpus import brown
from nltk.collocations import *
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk import bigrams
from nltk import collocations
from nltk import trigrams
from nltk.corpus import stopwords
from nltk.corpus import brown

## Start by loading the .csv files and setting the columns.
### For these .csv files, downloaded from https://github.com/ytsvetko/metaphor inputs and saved as .csv with the first row added as "sample"

In [44]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet.head()

,sample
0,angry welt
1,bald assertion
2,bare outline
3,black humor
4,blind alley


In [45]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet['metaphor'] = 1
df_anmet['an_met'] = 1
df_anmet['an_nonmet'] = 0
df_anmet['svo_met'] = 0
df_anmet['svo_nonmet'] = 0
df_anmet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,angry welt,1,1,0,0,0
1,bald assertion,1,1,0,0,0
2,bare outline,1,1,0,0,0
3,black humor,1,1,0,0,0
4,blind alley,1,1,0,0,0


In [46]:
df_annonmet = pd.read_csv("an_nonmets.csv", low_memory=False)
df_annonmet['metaphor'] = 0
df_annonmet['an_met'] = 0
df_annonmet['an_nonmet'] = 1
df_annonmet['svo_met'] = 0
df_annonmet['svo_nonmet'] = 0
df_annonmet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,angry protester,0,0,1,0,0
1,bald eagle,0,0,1,0,0
2,big city,0,0,1,0,0
3,blind man,0,0,1,0,0
4,bloody nose,0,0,1,0,0


In [47]:
df_svomet = pd.read_csv("svo_mets.csv", low_memory=False)
df_svomet['metaphor'] = 1
df_svomet['an_met'] = 0
df_svomet['an_nonmet'] = 0
df_svomet['svo_met'] = 1
df_svomet['svo_nonmet'] = 0
df_svomet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,conversation turn subject,1,0,0,1,0
1,resumption bring relief,1,0,0,1,0
2,economy move direction,1,0,0,1,0
3,service meet expectation,1,0,0,1,0
4,material live dream,1,0,0,1,0


In [48]:
df_svononmet = pd.read_csv("svo_nonmets.csv", low_memory=False)
df_svononmet['metaphor'] = 0
df_svononmet['an_met'] = 0
df_svononmet['an_nonmet'] = 0
df_svononmet['svo_met'] = 0
df_svononmet['svo_nonmet'] = 1
df_svononmet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,car break *none,0,0,0,0,1
1,crowd scream *none,0,0,0,0,1
2,*person turn *none,0,0,0,0,1
3,foot slip *none,0,0,0,0,1
4,man shake head,0,0,0,0,1


In [49]:
#combine into one df
frames = [df_anmet, df_annonmet, df_svomet, df_svononmet]
df_combo = pd.concat(frames)

## Break down into training, dev, and test sets

In [62]:
random_index = np.random.permutation(df_combo.index)
random_index[:10]
df_combo.ix[random_index, ['sample', 'metaphor', 'an_met', 'an_nonmet', 'svo_met', 'svo_nonmet']]
df_shuffled = df_combo.ix[random_index, ['sample', 'metaphor', 'an_met', 'an_nonmet', 'svo_met', 'svo_nonmet']]
df_shuffled.reset_index(drop=True, inplace=True)
df_shuffled[500:510]

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
500,*none shut brain,1,0,0,1,0
501,bus arrive *none,0,0,0,0,1
502,white anger,1,1,0,0,0
503,weak muscle,0,0,1,0,0
504,car complain *none,1,0,0,1,0
505,Frank stretch arm,0,0,0,0,1
506,hard truth,1,1,0,0,0
507,green vegetable,0,0,1,0,0
508,fortune smile *pronoun,1,0,0,1,0
509,patient tear cuff,0,0,0,0,1


In [65]:
rows, columns = df_shuffled.shape
print("Rows:", rows)
print("Columns:", columns)
#train_size = round(rows*.6)
train_size = round(rows*.9)
#dev_size   = round(rows*.2)
dev_size   = round(rows*.1)
df_train = df_shuffled.loc[:train_size]
df_train.shape
df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
df_dev.shape
df_test = df_shuffled.loc[dev_size+train_size:].reset_index(drop=True)
df_test.shape

Rows: 1644
Columns: 6


,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,raw emotion,1,1,0,0,0
1,large group,0,0,1,0,0
2,solution stare face,1,0,0,1,0
3,Miranda open mouth,0,0,0,0,1
4,mad sprint,1,1,0,0,0
5,hollow cylinder,0,0,1,0,0
6,bolt refuse *none,1,0,0,1,0
7,*person stare stone,0,0,0,0,1
8,narrow mind,1,1,0,0,0
9,icy road,0,0,1,0,0


## CountVectorizer
### Use count vecotrizer with df = 3 for unigram and bigrams to predict dev set

In [244]:
vocab = list(set(brown.words()))
print(vocab)

['Elector', 'keynote', 'quill', 'outdoor', 'docketed', 'earnings', 'disproportionately', 'sweetness', 'burns', 'middle-aged', 'Constantinople', 'stealer', 'Smithsonian', 'wreath', 'burnt', 'drone', 'hereinafter', 'Traditionalism', 'ABC', 'recently', 'Ranavan', 'resale', 'Sweet', '10:45', 'ballyhoo', 'not-less-deadly', '56A', 'Crime', 'Alabamas', 'Moroccan', 'Vast', 'near', 'Prometheus', 'bronzed', 'barium', 'Trout', 'Grubb', 'nearly', 'obey', 'truthful', 'Rastus', 'believeth', '4-7/8', 'Precinct', 'paycheck', 'trundling', 'one-sixteenth', 'years', 'banquet', 'amounted', 'platted', 'royalty-free', 'McNear', 'self-perceived', 'Sunman', 'anorthic', 'Pelham', 'mentioning', 'value-problems', 'reacts', 'hollyhocks', 'defenders', 'interlining', 'ca.', 'catchy', 'Cumhuriyet', 'Lafayette', 'impromptu', 'Crumley', 'convey', 'Cassius', '7%', 'unified', 'positivism', 'scop', 'producer', "mothers'", 'custodial', 'freighter', 'temerity', '$310', 'many', 'oneself', 'batteries', 'leavin', 'domestic', 

In [268]:
vec = CountVectorizer(ngram_range=(1, 3), token_pattern=r'\b\w+\b', analyzer=u'word', min_df=1, vocabulary=vocab)
df_train = df_train.fillna("")
df_dev = df_dev.fillna("")
df_test = df_test.fillna("")

In [269]:
arr_train_feature_sparse = vec.fit_transform(df_train['sample'])
arr_train_feature_sparse
arr_train_feature = arr_train_feature_sparse.toarray()
feature_labels = vec.get_feature_names()

In [270]:
arr_dev_feature_sparse = vec.transform(df_dev["sample"])
arr_dev_feature = arr_dev_feature_sparse.toarray()

In [271]:
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor']) #defining features (from reviews) and passing in Category label
logreg_predictions = logreg_model.predict(arr_dev_feature)
accuracy_score(df_dev['metaphor'], logreg_predictions)

0.95731707317073167

In [56]:
## make a file of sample sentences to practice on
## loop through sentence and .split()
## use nltk bigrams
##
## put into pandas as df_test

In [272]:
word_list = 'Jamal was a pig at dinner'.split()
filtered_words = [word for word in word_list if word not in stopwords.words('english')]
filtered_words
testmeta = list(trigrams(filtered_words))
testmeta
testmeta1 = [' '.join(x) for x in testmeta]
testmeta1
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample']
print(filtered_words)

['Jamal', 'pig', 'dinner']


In [273]:
arr_test_feature_sparse = vec.transform(df_test['sample'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
logreg_predictions

array([1])

In [274]:
df_test = pd.DataFrame(testmeta1)
df_test

,0
0,Jamal pig dinner


In [275]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in trigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5']
df_test = df_test.fillna("")
df_test

,sample0,sample1,sample2,sample3,sample4,sample5
0,The snow white,snow white blanket.,,,,
1,The hospital refrigerator.,,,,,
2,The classroom zoo.,,,,,
3,America melting pot.,,,,,
4,Her lovely voice,lovely voice music,voice music ears.,,,
5,Life roller coaster.,,,,,
6,The alligator’s teeth,alligator’s teeth white,teeth white daggers.,,,
7,Their home prison.,,,,,
8,The slide playground,slide playground hot,playground hot stove.,,,
9,His heart cold,heart cold iron.,,,,


In [276]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor5'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] 

if sum(df_test['sum_metaphor']) >= 1:
    df_test['sum_metaphor'] = 1
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,metaphor0,metaphor1,metaphor2,metaphor3,metaphor4,metaphor5,sum_metaphor
0,The snow white,snow white blanket.,,,,,0,0,1,1,1,1,1
1,The hospital refrigerator.,,,,,,1,1,1,1,1,1,1
2,The classroom zoo.,,,,,,1,1,1,1,1,1,1
3,America melting pot.,,,,,,1,1,1,1,1,1,1
4,Her lovely voice,lovely voice music,voice music ears.,,,,0,0,0,1,1,1,1
5,Life roller coaster.,,,,,,1,1,1,1,1,1,1
6,The alligator’s teeth,alligator’s teeth white,teeth white daggers.,,,,1,1,1,1,1,1,1
7,Their home prison.,,,,,,1,1,1,1,1,1,1
8,The slide playground,slide playground hot,playground hot stove.,,,,1,0,1,1,1,1,1
9,His heart cold,heart cold iron.,,,,,1,1,1,1,1,1,1


In [277]:
sum(df_test['sum_metaphor'])

71

In [278]:
with open("metanet.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in trigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
df_test = df_test.fillna("")
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,sample6
0,ability evaluate government,evaluate government ability,government ability see,,,,
1,ability evaluate ability,evaluate ability see,,,,,
2,ability know ability,know ability see,,,,,
3,abusive political leaders,political leaders physical,leaders physical bullies,,,,
4,,,,,,,
5,access education access,education access object,,,,,
6,access knowledge access,knowledge access object,,,,,
7,achieving purpose acquiring,purpose acquiring desired,acquiring desired object,,,,
8,achieving purpose reaching,purpose reaching destination,,,,,
9,achieving gun rights,gun rights reaching,rights reaching destination,,,,


In [279]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] 

if sum(df_test['sum_metaphor']) >= 1:
    df_test['sum_metaphor'] = 1
sum(df_test['sum_metaphor'])

1184

In [280]:
with open("an_nonmets.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in bigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample0']
df_test = df_test.fillna("")
df_test

arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

sum(df_test['metaphor0'])
#len(df_test['metaphor0'])

3

In [223]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in trigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
df_test = df_test.fillna("")
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,sample6
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words,meanings words arranged,words arranged across
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays",atlas displays rainbow,displays rainbow hues
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one,shows one specific,one specific aspect
3,No single brain,single brain region,brain region holds,region holds one,holds one word,one word concept.,
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related,number related words.,
5,And single word,single word lights,word lights many,lights many different,many different brain,different brain spots.,
6,Together make networks,make networks represent,networks represent meanings,represent meanings word,meanings word use:,word use: life,use: life love;
7,All light networks.,,,,,,
8,Described “tour de,“tour de force”,de force” one,force” one researcher,one researcher involved,"researcher involved study,","involved study, atlas"
9,"With advances, technology","advances, technology could",technology could profound,could profound impact,profound impact medicine,impact medicine fields.,


In [224]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] 

if sum(df_test['sum_metaphor']) >= 1:
    df_test['sum_metaphor'] = 1
sum(df_test['sum_metaphor'])
#len(df_test['sum_metaphor'])
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,sample6,metaphor0,metaphor1,metaphor2,metaphor3,metaphor4,metaphor5,metaphor6,sum_metaphor
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words,meanings words arranged,words arranged across,1,1,1,1,1,1,1,1
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays",atlas displays rainbow,displays rainbow hues,1,1,1,1,1,1,1,1
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one,shows one specific,one specific aspect,1,1,1,1,1,1,1,1
3,No single brain,single brain region,brain region holds,region holds one,holds one word,one word concept.,,1,1,1,1,1,1,1,1
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related,number related words.,,1,1,1,0,0,0,1,1
5,And single word,single word lights,word lights many,lights many different,many different brain,different brain spots.,,1,1,1,1,1,1,1,1
6,Together make networks,make networks represent,networks represent meanings,represent meanings word,meanings word use:,word use: life,use: life love;,1,1,1,1,1,1,1,1
7,All light networks.,,,,,,,1,1,1,1,1,1,1,1
8,Described “tour de,“tour de force”,de force” one,force” one researcher,one researcher involved,"researcher involved study,","involved study, atlas",1,1,1,1,1,1,1,1
9,"With advances, technology","advances, technology could",technology could profound,could profound impact,profound impact medicine,impact medicine fields.,,1,1,1,1,1,1,1,1


In [176]:
df_test.ix[:, 0:5].head()

,0,1,2,3,4,5
0,The snow white,snow white blanket.,,,,
1,The hospital refrigerator.,,,,,
2,The classroom zoo.,,,,,
3,America melting pot.,,,,,
4,Her lovely voice,lovely voice music,voice music ears.,,,


In [179]:
len(df_test.columns)
#df_test[0]

6

In [182]:
for j in range(0, len(df_test.columns)-1):
    arr_test_feature_sparse = vec.transform(df_test[j])
    arr_test_feature = arr_test_feature_sparse.toarray()
    logreg = LogisticRegression()
    logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
    logreg_predictions = logreg_model.predict(arr_test_feature)
    indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
    label = 'metaphor' + str(j)
    df_test[label] = logreg_predictions

# arr_test_feature_sparse = vec.transform(df_test['sample0'])
# arr_test_feature = arr_test_feature_sparse.toarray()
# logreg = LogisticRegression()
# logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
# logreg_predictions = logreg_model.predict(arr_test_feature)
# indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor0'] = logreg_predictions

# arr_test_feature_sparse = vec.transform(df_test['sample1'])
# arr_test_feature = arr_test_feature_sparse.toarray()
# logreg = LogisticRegression()
# logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
# logreg_predictions = logreg_model.predict(arr_test_feature)
# indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor1'] = logreg_predictions

# arr_test_feature_sparse = vec.transform(df_test['sample2'])
# arr_test_feature = arr_test_feature_sparse.toarray()
# logreg = LogisticRegression()
# logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
# logreg_predictions = logreg_model.predict(arr_test_feature)
# indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor2'] = logreg_predictions

# arr_test_feature_sparse = vec.transform(df_test['sample3'])
# arr_test_feature = arr_test_feature_sparse.toarray()
# logreg = LogisticRegression()
# logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
# logreg_predictions = logreg_model.predict(arr_test_feature)
# indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor3'] = logreg_predictions

# arr_test_feature_sparse = vec.transform(df_test['sample4'])
# arr_test_feature = arr_test_feature_sparse.toarray()
# logreg = LogisticRegression()
# logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
# logreg_predictions = logreg_model.predict(arr_test_feature)
# indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor4'] = logreg_predictions

# arr_test_feature_sparse = vec.transform(df_test['sample5'])
# arr_test_feature = arr_test_feature_sparse.toarray()
# logreg = LogisticRegression()
# logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
# logreg_predictions = logreg_model.predict(arr_test_feature)
# indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor5'] = logreg_predictions

#df_test['sum_metaphor'] = df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] 

# if sum(df_test['sum_metaphor']) >= 1:
#     df_test['sum_metaphor'] = 1
# df_test

KeyError: 6